In [1]:
%matplotlib inline

import itertools
import numpy as np
import os
import random
import sys
import psutil
import tensorflow as tf
from PIL import Image
import matplotlib.pyplot as plt
plt.switch_backend('agg')

if "./lib" not in sys.path:
    sys.path.append("./lib")
    
import plotting
from collections import deque, namedtuple
from readingFileEfficiently import *
import VOC2012_npz_files_writter
from DNN import *
from Agent import ObjLocaliser

/home/mohammad/Tensorflow/local/lib/python2.7/site-packages/requests/__init__.py:83: RequestsDependencyWarning: Old version of cryptography ([1, 2, 3]) may cause slowdown.
  warnings.warn(warning, RequestsDependencyWarning)


In [2]:
#This cell reads VOC 2012 dataset and save them in .npz files for future.
#The process of reading data and put them in prper format is time consuming so they are stored in a file.

destination = "../data/"

#It splits dataset to 80% for training and 20% validation.
if not (os.path.isfile(destination+"test_input.npz") or os.path.isfile(destination+"test_target.npz")):
    print("Files are not ready!!!")
else:
    print("Records are already prepared!!!")

Records are already prepared!!!


In [ ]:
num_episodes = 15     
category = "cat"

model_name = "cat_loss_test"


tf.reset_default_graph()

# Where we save our checkpoints and graphs
experiment_dir = os.path.abspath("/home/mohammad/active localization/trained_on_cluster_final/{}".format(model_name))

# Create a glboal step variable
global_step = tf.Variable(0, name='global_step', trainable=False)
    
# Create estimators
q_estimator = Estimator(scope="q_estimator", summaries_dir=experiment_dir)

# State processor
state_processor = StateProcessor()



with tf.Session() as sess:
    
    sess.run(tf.global_variables_initializer()) 
    num_located = 0

    # For 'system/' summaries, usefull to check if currrent process looks healthy
    current_process = psutil.Process()

    # Create directories for checkpoints and summaries
    checkpoint_dir = os.path.join(experiment_dir, "checkpoints")
    checkpoint_path = os.path.join(checkpoint_dir, "model")
    report_path = os.path.join(experiment_dir, "report")

    if not os.path.exists(checkpoint_dir):
        os.makedirs(checkpoint_dir)
    if not os.path.exists(report_path):
        os.makedirs(report_path)
    saver = tf.train.Saver()
    # Load a previous checkpoint if we find one
    latest_checkpoint = tf.train.latest_checkpoint(checkpoint_dir)
    if latest_checkpoint:
        print("Loading model checkpoint {}...\n".format(latest_checkpoint))
        saver.restore(sess, latest_checkpoint)

    # Get the current time step
    total_t = sess.run(tf.contrib.framework.get_global_step())


    # The policy we're following
    policy = make_epsilon_greedy_policy(
        q_estimator,
        len(VALID_ACTIONS))

    precisions = []
    
    for indx,tmp in enumerate(extractData(category, "test", 32)):


        
        img=tmp[0]
        target=tmp[1]
        succ = 0

        im2 = Image.frombytes("RGB",(img['image_width'],img['image_height']),img['image'])
        env = ObjLocaliser(np.array(im2),target)
        print "New image is being loaded: {}".format(img['image_filename'])
            
    
        for i_episode in range(num_episodes):

            # Save the current checkpoint
            saver.save(tf.get_default_session(), checkpoint_path)

            # Reset the environment
            env.Reset(np.array(im2))
            state = env.wrapping()
            state = state_processor.process(sess, state)
            state = np.stack([state] * 4, axis=2)

            t=0
            action = 0
            
            # One step in the environment
            while (action != 10) and (t < 50):



                action_probs, qs = policy(sess, state, 0.2)
                #print action_probs
                #print qs
                action = np.random.choice(np.arange(len(action_probs)), p=action_probs)
                reward = env.takingActions(VALID_ACTIONS[action])
                
                if reward == 3:
                    succ += 1
                
                #env.drawActions()
                next_state = env.wrapping()
                next_state = state_processor.process(sess, next_state)
                next_state = np.append(state[:,:,1:], np.expand_dims(next_state, 2), axis=2)
                state = next_state
                
                t += 1
                
            print "number of actions for step {} is: {}".format(i_episode, t)

        
        precisions.append(float(succ)/num_episodes)
        print "image {} precision: {}".format(img['image_filename'], precisions[-1])



Instructions for updating:
Please switch to tf.train.get_global_step


/home/mohammad/Tensorflow/local/lib/python2.7/site-packages/tensorflow/python/ops/gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Loading model checkpoint /home/mohammad/active localization/trained_on_cluster_final/cat_loss_test/checkpoints/model...

INFO:tensorflow:Restoring parameters from /home/mohammad/active localization/trained_on_cluster_final/cat_loss_test/checkpoints/model
input file is loading...
New image is being loaded: 2008_001836.jpg
number of actions for step 0 is: 50
number of actions for step 1 is: 50
number of actions for step 2 is: 50
number of actions for step 3 is: 50
number of actions for step 4 is: 16
number of actions for step 5 is: 18
number of actions for step 6 is: 50
number of actions for step 7 is: 50
number of actions for step 8 is: 7
number of actions for step 9 is: 50
number of actions for step 10 is: 12
number of actions for step 11 is: 2
number of actions for step 12 is: 50
number of actions for step 13 is: 50
number of actions for step 14 is: 6
image 2008_001836.jpg precision: 0.0
New image is being loaded: 2009_004999.jpg
number of actions for step 0 is: 24
number of actions f

number of actions for step 11 is: 50
number of actions for step 12 is: 20
number of actions for step 13 is: 50
number of actions for step 14 is: 50
image 2008_002201.jpg precision: 0.0666666666667
New image is being loaded: 2007_002445.jpg
number of actions for step 0 is: 27
number of actions for step 1 is: 50
number of actions for step 2 is: 25
number of actions for step 3 is: 21
number of actions for step 4 is: 15
number of actions for step 5 is: 50
number of actions for step 6 is: 14
number of actions for step 7 is: 47
number of actions for step 8 is: 11
number of actions for step 9 is: 50
number of actions for step 10 is: 3
number of actions for step 11 is: 44
number of actions for step 12 is: 27
number of actions for step 13 is: 50
number of actions for step 14 is: 26
image 2007_002445.jpg precision: 0.133333333333
New image is being loaded: 2009_000002.jpg
number of actions for step 0 is: 50
number of actions for step 1 is: 38
number of actions for step 2 is: 50
number of actions

number of actions for step 12 is: 11
number of actions for step 13 is: 23
number of actions for step 14 is: 8
image 2008_000222.jpg precision: 0.466666666667
New image is being loaded: 2010_001030.jpg
number of actions for step 0 is: 21
number of actions for step 1 is: 50
number of actions for step 2 is: 50
number of actions for step 3 is: 23
number of actions for step 4 is: 40
number of actions for step 5 is: 48
number of actions for step 6 is: 4
number of actions for step 7 is: 21
number of actions for step 8 is: 50
number of actions for step 9 is: 47
number of actions for step 10 is: 9
number of actions for step 11 is: 50
number of actions for step 12 is: 35
number of actions for step 13 is: 13
number of actions for step 14 is: 7
image 2010_001030.jpg precision: 0.133333333333
New image is being loaded: 2009_004358.jpg
number of actions for step 0 is: 9
number of actions for step 1 is: 3
number of actions for step 2 is: 44
number of actions for step 3 is: 8
number of actions for ste

...

number of actions for step 2 is: 50
number of actions for step 3 is: 26
number of actions for step 4 is: 25
number of actions for step 5 is: 4
number of actions for step 6 is: 50
number of actions for step 7 is: 50
number of actions for step 8 is: 14
number of actions for step 9 is: 12
number of actions for step 10 is: 7
number of actions for step 11 is: 12
number of actions for step 12 is: 5
number of actions for step 13 is: 20
number of actions for step 14 is: 40
image 2010_004634.jpg precision: 0.133333333333
New image is being loaded: 2010_004457.jpg
number of actions for step 0 is: 50
number of actions for step 1 is: 50
number of actions for step 2 is: 50
number of actions for step 3 is: 50
number of actions for step 4 is: 50
number of actions for step 5 is: 41
number of actions for step 6 is: 50
number of actions for step 7 is: 45
number of actions for step 8 is: 37
number of actions for step 9 is: 11
number of actions for step 10 is: 7
number of actions for step 11 is: 9
number 

number of actions for step 2 is: 27
number of actions for step 3 is: 50
number of actions for step 4 is: 10
number of actions for step 5 is: 50
number of actions for step 6 is: 50
number of actions for step 7 is: 16
number of actions for step 8 is: 50
number of actions for step 9 is: 50
number of actions for step 10 is: 50
number of actions for step 11 is: 40
number of actions for step 12 is: 50
number of actions for step 13 is: 50
number of actions for step 14 is: 40
image 2011_001841.jpg precision: 0.2
New image is being loaded: 2010_004585.jpg
number of actions for step 0 is: 28
number of actions for step 1 is: 50
number of actions for step 2 is: 50
number of actions for step 3 is: 50
number of actions for step 4 is: 10
number of actions for step 5 is: 10
number of actions for step 6 is: 50
number of actions for step 7 is: 9
number of actions for step 8 is: 50
number of actions for step 9 is: 12
number of actions for step 10 is: 50
number of actions for step 11 is: 50
number of acti

number of actions for step 4 is: 50
number of actions for step 5 is: 26
number of actions for step 6 is: 50
number of actions for step 7 is: 50
number of actions for step 8 is: 50
number of actions for step 9 is: 1
number of actions for step 10 is: 11
number of actions for step 11 is: 50
number of actions for step 12 is: 11
number of actions for step 13 is: 46
number of actions for step 14 is: 50
image 2011_001128.jpg precision: 0.333333333333
New image is being loaded: 2007_009794.jpg
number of actions for step 0 is: 50
number of actions for step 1 is: 50
number of actions for step 2 is: 36
number of actions for step 3 is: 50
number of actions for step 4 is: 3
number of actions for step 5 is: 36
number of actions for step 6 is: 50
number of actions for step 7 is: 2
number of actions for step 8 is: 50
number of actions for step 9 is: 50
number of actions for step 10 is: 50
number of actions for step 11 is: 27
number of actions for step 12 is: 50
number of actions for step 13 is: 50
num

number of actions for step 5 is: 38
number of actions for step 6 is: 29
number of actions for step 7 is: 26
number of actions for step 8 is: 7
number of actions for step 9 is: 43
number of actions for step 10 is: 20
number of actions for step 11 is: 10
number of actions for step 12 is: 3
number of actions for step 13 is: 26
number of actions for step 14 is: 11
image 2010_002909.jpg precision: 0.133333333333
New image is being loaded: 2008_005247.jpg
number of actions for step 0 is: 46
number of actions for step 1 is: 13
number of actions for step 2 is: 37
number of actions for step 3 is: 20
number of actions for step 4 is: 50
number of actions for step 5 is: 50
number of actions for step 6 is: 50
number of actions for step 7 is: 1
number of actions for step 8 is: 50
number of actions for step 9 is: 50
number of actions for step 10 is: 48
number of actions for step 11 is: 50
number of actions for step 12 is: 20
number of actions for step 13 is: 4
number of actions for step 14 is: 11
ima

number of actions for step 7 is: 34
number of actions for step 8 is: 46
number of actions for step 9 is: 43
number of actions for step 10 is: 29
number of actions for step 11 is: 39
number of actions for step 12 is: 50
number of actions for step 13 is: 33
number of actions for step 14 is: 2
image 2010_006025.jpg precision: 0.0666666666667
New image is being loaded: 2008_002749.jpg
number of actions for step 0 is: 50
number of actions for step 1 is: 50
number of actions for step 2 is: 45
number of actions for step 3 is: 13
number of actions for step 4 is: 50
number of actions for step 5 is: 11
number of actions for step 6 is: 4
number of actions for step 7 is: 16
number of actions for step 8 is: 21
number of actions for step 9 is: 20
number of actions for step 10 is: 28
number of actions for step 11 is: 30
number of actions for step 12 is: 50
number of actions for step 13 is: 19
number of actions for step 14 is: 47
image 2008_002749.jpg precision: 0.0666666666667
New image is being load

In [ ]:
len(precisions)

In [ ]:
np.mean(precisions)

In [ ]:
b = [1 if i > 0 else 0 for i in precisions ]
np.mean(b)